In [41]:
!pip install plotly cufflinks

  Running setup.py bdist_wheel for cufflinks ... - \ done
  Stored in directory: /home/jovyan/.cache/pip/wheels/de/a9/69/f853eccf2c2ac3ba8e7c79f904eedf354fea15d136902cd8b6
  Running setup.py bdist_wheel for colorlover ... - \ done
  Stored in directory: /home/jovyan/.cache/pip/wheels/b8/b0/18/76d3f3088cc73950ed1aa44ea074b93ed37309fea57dec78dc
Successfully built cufflinks colorlover
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Exploration") \
    .getOrCreate()

In [3]:
flights = spark.read.parquet('/data/parquet/flights')

In [4]:
flights.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1988|    1|        19|        2|   2130|      2130|   2234|      2231|           PS|     1823

In [4]:
flights.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [5]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
import pandas as pd

In [7]:
flights.limit(100).toPandas()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1988,1,19,2,2130,2130,2234,2231,PS,1823,...,NA,NA,0,NA,0,NA,NA,NA,NA,NA
1,1988,1,20,3,2139,2140,2242,2246,PS,1823,...,NA,NA,0,NA,0,NA,NA,NA,NA,NA
2,1988,1,21,4,2141,2140,2250,2246,PS,1823,...,NA,NA,0,NA,0,NA,NA,NA,NA,NA
3,1988,1,22,5,2143,2140,2242,2246,PS,1823,...,NA,NA,0,NA,0,NA,NA,NA,NA,NA
4,1988,1,24,7,2143,2140,2243,2246,PS,1823,...,NA,NA,0,NA,0,NA,NA,NA,NA,NA
5,1988,1,25,1,2143,2140,2245,2246,PS,1823,...,NA,NA,0,NA,0,NA,NA,NA,NA,NA
6,1988,1,26,2,2143,2140,2242,2246,PS,1823,...,NA,NA,0,NA,0,NA,NA,NA,NA,NA
7,1988,1,27,3,2142,2140,2241,2246,PS,1823,...,NA,NA,0,NA,0,NA,NA,NA,NA,NA
8,1988,1,28,4,2141,2140,2239,2246,PS,1823,...,NA,NA,0,NA,0,NA,NA,NA,NA,NA
9,1988,1,29,5,2143,2140,2247,2246,PS,1823,...,NA,NA,0,NA,0,NA,NA,NA,NA,NA


In [ ]:
table = ff.create_table(flights.limit(100).toPandas())
py.iplot(table, filename='table1')

In [25]:
flights = flights.withColumn('Delayed', flights.ArrDelay > 20)
print("%% delayed/(total): %.02f" % (flights.where(flights.Delayed == True).count() / flights.count()))

% delayed/(total): 0.15


# Flights by Year

In [76]:
import pyspark.sql.functions as func

delayedByYear = flights \
    .groupBy(flights.Year) \
    .pivot("Delayed", ['True', 'False']) \
    .count() \
    .toPandas()
delayedByYear['Total'] = delayedByYear['True'] + delayedByYear['False']
delayedByYear['Delayed'] = delayedByYear['True'] / delayedByYear['Total']
delayedByYear['OnTime'] = delayedByYear['False'] / delayedByYear['Total']
delayedByYear

,Year,True,False,Total,Delayed,OnTime
0,1987,212581,1066612,1279193,0.166184,0.833816
1,1988,668707,4459568,5128275,0.130396,0.869604
2,1989,792093,4151023,4943116,0.160242,0.839758


In [78]:
import plotly.plotly as py
import cufflinks as cf
import pandas as pd
import numpy as np

delayedByYear[['Year', 'Delayed', 'OnTime']].iplot(x='Year', kind='bar', barmode='stack', filename='pandas-bar-chart')

# Flights by Month

In [6]:
delayedByMonth = flights \
    .groupBy(flights.Month) \
    .count() \
    .toPandas()
delayedByMonth

,Month,count
0,7,858284
1,11,1261485
2,3,880261
3,8,872854
4,5,852076
5,6,837592
6,9,839143
7,1,858323
8,10,1318159
9,4,832929


In [7]:
import plotly.graph_objs as go

In [46]:
delayedByMonth.iplot(data, filename='pandas-bar-chart')

In [18]:
url

'https://plot.ly/~geraud/3'